In [1]:
import numpy as np
import copy

def evaluate_board(board, player):
    score = 0
    opponent = 1 if player == 2 else 2
    for i in range(19):
        for j in range(19):
            if j <= 14:
                pattern = board[i, j:j + 5]
                score += get_score_from_pattern(pattern, player, opponent)
            if i <= 14:
                pattern = board[i:i + 5, j]
                score += get_score_from_pattern(pattern, player, opponent)
            if i <= 14 and j <= 14:
                pattern = np.diag(board[i:i + 5, j:j + 5])
                score += get_score_from_pattern(pattern, player, opponent)
            if i <= 14 and j >= 4:
                pattern = np.diag(np.fliplr(board[i:i + 5, j - 4:j + 1]))
                score += get_score_from_pattern(pattern, player, opponent)
    return score

def get_score_from_pattern(pattern, player, opponent):
    score = 0
    player_count = pattern.tolist().count(player)
    opponent_count = pattern.tolist().count(opponent)

    if opponent_count == 0:
        if player_count == 5:
            score += 100000
        elif player_count == 4:
            score += 10000
        elif player_count == 3:
            score += 1000
        elif player_count == 2:
            score += 100
        elif player_count == 1:
            score += 10
    
    elif player_count == 0:
        if opponent_count == 5:
            score -= 150000
        elif opponent_count == 4:
            score -= 15000 
        elif opponent_count == 3:
            score -= 1500
        elif opponent_count == 2:
            score -= 150   
        elif opponent_count == 1:
            score -= 15 

    return score


def get_mini_max_move(board):
    if np.sum(board) == 0:
        return (9, 9)

    able_positions = set()
    directions = [(1, 0), (0, 1), (1, 1), (1, -1), (-1, 0), (0, -1), (-1, -1), (-1, 1)]

    for i in range(19):
        for j in range(19):
            if board[i, j] != 0:
                for dx, dy in directions:
                    for dist in range(1, 2):
                        nx, ny = i + dx * dist, j + dy * dist
                        if 0 <= nx < 19 and 0 <= ny < 19 and board[nx, ny] == 0:
                            able_positions.add((nx, ny))

    best_score = float('-inf')
    best_move = None

    for x, y in able_positions:
        new_board = copy.deepcopy(board)
        new_board[x, y] = 1
        score1 = evaluate_board(new_board, 1)

        able_positions2 = set()
        for i in range(19):
            for j in range(19):
                if new_board[i, j] != 0:
                    for dx, dy in directions:
                        for dist in range(1, 2):
                            nx, ny = i + dx * dist, j + dy * dist
                            if 0 <= nx < 19 and 0 <= ny < 19 and new_board[nx, ny] == 0:
                                able_positions2.add((nx, ny))

        score2 = 0        
        for nx, ny in able_positions2:
            new_board2 = copy.deepcopy(new_board)
            new_board2[nx, ny] = 2
            score2 = min(score2, evaluate_board(new_board2, 2))
            
        score = score1 + score2
        if score > best_score:
            best_score = score
            best_move = (x, y)

    return best_move



if __name__ == "__main__":
    import numpy as np

    # 테스트용 빈 보드 생성
    board = np.zeros((19, 19), dtype=int)
    #board[10,10] = 1
    
    # get_mini_max_move 함수 실행
    move = get_mini_max_move(board)
    print("Best move:", move)

Best move: (9, 9)


In [1]:
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

2025-05-13 19:39:39.187170: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 19:39:39.463981: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 19:39:40.305791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 19:39:40.308213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 19:39:43.677145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
model = load_model('/home/jinjinjara1022/gomoku/backend/app/strategies/model_weight/best.h5')

2025-05-13 19:39:50.686892: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
grid = np.zeros((19, 19), dtype=int)
input_ = np.expand_dims(grid, axis=(0, -1)).astype(np.float32)
output = model.predict(input_, verbose=0).squeeze()
output = output.reshape((19, 19))
output_y, output_x = np.unravel_index(np.argmax(output), output.shape)

In [4]:
print(output_y, output_x)

9 9
